## 基于 TorchServe 部署实验


## TorchServe 介绍

Torchserve 是 Facebook 公司开发的在线深度学习模型部署框架，它可以很方便的部署 Pytorch 的深度学习模型。GitHub 地址：https://github.com/pytorch/serve/blob/master/docs/README.md

### 1、模型从训练到部署的流程

<img src="./img/torchserve-tuning-flow-chart.png" width=600px />


- 模型训练：见上一课
- 基于硬件模型优化（可选）
  - 需要很深的编译器和并行计算的知识
  - 一般很大型的系统才用得上，都会由专门的团队来做
  - 感兴趣的同学可以参考[TVM](https://tvm.apache.org/)和[Optimum](https://github.com/huggingface/optimum)
- 配置模型部署框架：
  - **目的**：提供高可用的服务，使你的模型变成 Model-as-a-Service
  - **功能**：多进程（卡）间的负载均衡
  - **功能**：单卡上的动态 Batch：平衡延时与吞吐量
  - **功能**：模型与资源状态的实时监控等
  - 它与云中间件并不矛盾，可作为一个服务由云中间件统一管理
- 部署你的服务：打包容器，配置 k8s，...


### 模型的打包与部署

实验前提说明：

- 以 chatglm2-6b 作为实验模型
- 实验中模型 `chatglm2-6b` 路径为：`/root/chatglm2-6b`
- 实验用到的代码文件都在当前路径下的 `code` 文件夹里面

注意：

- 必须在具有 GPU 的机器上执行以下实验，下面实验在 AutoDL 上进行


### 0. 建立工程项目

- 在 ~/autodl-tmp 建立一个工程项目名字为：`torchserve-lab`
- `~/autodl-tmp` 是 AutoDL 产品独有的目录，用于数据挂在的目录，由于后期操作模型数据可能会比较大，系统盘不能扩容，我们只能用这个目录。
- 后面所有实验操作都在这个工程目录下面执行

```bash
cd ~/autodl-tmp && mkdir torchserve-lab
```

然后把当前课件下的 code 文件夹里面的代码，复制到工程项目`torchserve-lab`

安装环境

```
pip install torch torchvision
```


### 1. 将 chatglm2-6b 模型合并为 .bin 文件

- 如果你的模型路径与实验不一致，需要把下面命令中的 `/root/chatglm2-6b` 改成自己模型对应的路径
- 执行完命令后，会在工程目录下产生一个 `model` 文件夹

进入工程项目里面，然后把 code 文件夹里面的`load_and_save.py` 放到工程目录 `torchserve-lab` 里面

```bash
python load_and_save.py --model_name_or_path /root/chatglm2-6b --quantization_bit 4 --cache_dir ./model
```

model 文件内生成的文件结果，参考下图

<img src="./img/model-list.png" alt="模型列表" width="30%" height="30%">


### 2. 使用 `torch-model-archiver` 工具打包模型

安装 `torch-model-archiver` 和 `torchserve`：

```bash
pip install torch-model-archiver torchserve
```

把工程项目 code 文件里面的 `chatglm_handler.py`
使用 `torch-model-archiver` 打包模型：

注意：此时需要切换到工程项目下 model 文件里面执行。

```bash
cd model
```

```bash
torch-model-archiver --model-name chatglm2 \
    --version 1.0 \
    --serialized-file pytorch_model.bin \
    --handler ../chatglm_handler.py \
    --extra-files "config.json,configuration_chatglm.py,generation_config.json,modeling_chatglm.py,quantization.py,special_tokens_map.json,tokenization_chatglm.py,tokenizer_config.json,tokenizer.model"
```

这将在当前目录生成一个名为 `chatglm2.mar` 的文件。

<img src="./img/model-mar.png" alt="增加模型mar文件" width="30%" height="30%">

#### 参数说明：

- `--model-name`：模型名称；导出的模型文件将命名为“模型名称.mar”。
- `--serialized-file`：模型序列化文件；有两种类型的数据文件：
  - 包含状态字典的 Eager 模式文件（.pt/.pth/.bin 文件）
  - 在 TorchScript 条件下可执行的模块
- `--model-file`：（可选）模型结构框架，通常包含一个继承自`torch.nn.modules`的子类。
- `--handler`：TorchServer 的入口程序（见下文详细说明）。
- `--extra-files`：附加文件，可以包括多个文件并连接成一个字符串。
  - 对于 Hugging Face 模型，通常包括所有模型加载所需的文件。
- `--run-time`：（可选）选择运行的 Python 版本。
- `--archive-format`：（可选）选择文件的压缩格式：{tgz，no-archive，default}。
  - 可以是 tgz 压缩文件或 mar 文件。
- `--export-path`：（可选）保存 mar 存档文件的目标位置；默认为当前目录。
- `-f`：强制覆盖。
- `-v --version`：模型版本。
- `-r, --requirement-f`：（可选）模型环境相关的依赖包在`requirements.txt`中。

<b>`--handler`说明：</b> TorchServer 的入口程序。


### 3. 启动和停止 TorchServe 服务

回到工程的根目录

```
cd ~/autodl-tmp/torchserve-lab
```

#### 启动 TorchServe：

首先，创建一个目录来存储模型：

```bash
mkdir model_store
```

然后，启动 TorchServe：

`config.properties` 这个文件放在了当前路径下的 code 文件夹里面

```bash
torchserve --start --model-store model_store --models chatglm2.mar --ts-config config.properties
```

其中 `model_store` 是存储 `.mar` 文件的目录。

启动后会出现如下图所示，此时不能确保是否成功，参考后面教程，来验证一下是否成功。

![TorchServe 运行](./img/TorchServe-run.png)

注意！！

如果出现：`No space left on device` 说明磁盘空间不够了，需要去扩容了

![no space](./img/no-space.png)


### 验证并且查询已经注册的模型

当我们执行完下面命令后，会出现模型列表，代表 `TorchServe` 成功运行

```
curl "http://localhost:8881/models"
```

![模型列表](./img/modes-list-ok.png)

注意！！！

当端口出现 `refused` 的代表，服务还没启动成功或者启动失败，这时候就需要通过日志去排查问题了

![refused](./img/refused.png)


### 测试 API 接口返回数据

```bash
curl http://127.0.0.1:6006/predictions/chatglm2 \
     -X POST \
     -H "Content-Type: application/json" \
     -d '{"context":"你好"}'

```

![模型数据测试成功](./img/model-data-ok.png)

### 通过外网接口访问

同样，我们可以参考之前的经验，点击 AutoDL 的自定义服务，访问外网服务。

![自定义配置](./img/diy-config.png)

把跳转后的 url 地址替换上面的 `http://127.0.0.1:6006` 即可

![curl服务](./img/curl-data.png)


### 停止 TorchServe：

```bash
torchserve --stop
```

以上就是将 PyTorch 模型转换为 TorchServe 可用格式，使用 `torch-model-archiver` 工具，并启动和停止 TorchServe 服务的过程。
